<font color='navy'> <strong> Importamos a biblioteca Pandas, que facilita a manipulação de dados, depois lemos a planilha original. 

É preciso também inserir os zeros à esquerda para compor os 14 dígitos do cnpj dos postos.

Abaixo é possível ver uma amostra do resultado trazendo os primeiros 5 registros. </font></strong>


In [217]:
import pandas as pd

dfOriginal = pd.read_csv('combustiveis-original.csv')


#Transforma o campo cnpj em string e preenche de zeros até dar os 14 dígitos
dfOriginal['CNPJ'] = (dfOriginal['CNPJ']).astype(str).copy()
dfOriginal['CNPJ'] = dfOriginal['CNPJ'].apply(lambda x: x.zfill(14))

dfOriginal.head(5)

,CNPJ,DSC_PRODUTO,DIA,MES,ANO,VAL_MEDIO_VENDIDO
0,12519476000112,COMB GASOLINA COMUM Bico 07,23,7,2019,4.358588
1,12519476000112,COMB GASOLINA COMUM Bico 15,22,7,2019,4.355683
2,12519476000112,COMB GASOLINA COMUM Bico 15,28,7,2019,4.335388
3,12519476000112,COMB GASOLINA COMUM Bico 16,16,7,2019,4.343486
4,12519476000112,COMB GASOLINA COMUM Bico 16,24,7,2019,4.345863


<font color='navy'> <strong> Abaixo foi preciso unir todas as vendas de gasolina resumindo resultado pela média das vendas de cada cnpj.

Obs.
Procuramos isolar apenas as vendas de gasolina comum, mas identificamos que 27 postos no estado venderam apenas gasolina aditivada no período. Entendemos que o mais justo seria fazer a média de todos os tipos de gasolina vendidos em cada posto, e assim foi feito. </font></strong>


In [219]:
dfOriginal = dfOriginal.groupby('CNPJ')[['VAL_MEDIO_VENDIDO']].mean().copy()
dfOriginal.head(5)

,VAL_MEDIO_VENDIDO
CNPJ,
00404850000155,4.209992
00404850000236,4.173989
00404850000317,4.385104
00404850000406,4.754319
00440239000182,4.457878


<font color='navy'> <strong> É preciso observar que originalmente a planilha que nos foi fornecida pela Sefaz não traz outras informações sobre os postos exceto seu CNPJ.
 
 A partir daqui foi preciso montar uma grande query (consulta), para conseguir obter outras informações a respeito dos postos de combustível, como nome, cidade e bairro onde se localizam. Essa consulta foi feita a uma grande base de dados fornecida pela Receita Federal onde estão listadas todas as empresas do país.</font></strong>

In [239]:
#Monta a query a partir dos CNPJs

query = ("SELECT cnpj, razao_social , nome_fantasia, bairro, cod_municipio, municipio FROM empresas WHERE (cnpj LIKE ")

for index, row in dfOriginal.iterrows():
    query = query + "'" + index + "'"
    query = query + " OR cnpj LIKE "

#Remove o último: " OR cnpj LIKE "
query = query[:-14]
query = query + ")"
print(query)

SELECT cnpj, razao_social , nome_fantasia, bairro, cod_municipio, municipio FROM empresas WHERE (cnpj LIKE '00404850000155' OR cnpj LIKE '00404850000236' OR cnpj LIKE '00404850000317' OR cnpj LIKE '00404850000406' OR cnpj LIKE '00440239000182' OR cnpj LIKE '00497402000143' OR cnpj LIKE '00666880000130' OR cnpj LIKE '00740510000103' OR cnpj LIKE '00807244000261' OR cnpj LIKE '00994685000139' OR cnpj LIKE '01242690000158' OR cnpj LIKE '01242690000239' OR cnpj LIKE '01242690000310' OR cnpj LIKE '01332922000169' OR cnpj LIKE '01600494000108' OR cnpj LIKE '01774803000166' OR cnpj LIKE '02379869000548' OR cnpj LIKE '02418116000170' OR cnpj LIKE '02448145000185' OR cnpj LIKE '02485626000160' OR cnpj LIKE '02485967000136' OR cnpj LIKE '02488031000169' OR cnpj LIKE '02496680000101' OR cnpj LIKE '02681327000100' OR cnpj LIKE '02804864000191' OR cnpj LIKE '02817655000182' OR cnpj LIKE '02850879000196' OR cnpj LIKE '02850879000277' OR cnpj LIKE '02957259000150' OR cnpj LIKE '02970015000108' OR cnp

In [240]:
#Importa a biblioteca para acesso ao banco de dados e executa a query montada anteriormente
#Esse é o processo onde efetivamente é feita a consulta ao banco de dados, e leva alguns minutos para retornar os resultados.
#O banco de dados foi baixado do site da RFB e precisou passar por um processo anterior para ser convertido

import sqlite3

conn = sqlite3.connect('CNPJ_full.db')
cursor = conn.cursor()

cursor.execute(query)
 
rows = cursor.fetchall()
 


In [241]:
#Define quais são os campos desse novo dataframe obtido junto à RFB

dfReceita = pd.DataFrame(rows)
dfReceita.columns = ['CNPJ', 'RAZAO_SOCIAL', 'NOME_FANTASIA', 'BAIRRO', 'COD_MUNICIPIO', 'MUNICIPIO']

#SE NÃO TIVER NOME FANTASIA RECEBE A RAZÃO SOCIAL NO LUGAR
for index, row in dfReceita.iterrows():    
    fantasia = (row['NOME_FANTASIA'])
    
    if row['NOME_FANTASIA'] == "":        
        row['NOME_FANTASIA'] = row['RAZAO_SOCIAL']        
        dfReceita['NOME_FANTASIA'][index] =  dfReceita['RAZAO_SOCIAL'][index]

#Deixa o campo cnpj desse novo dataframe no formato de string, assim ele passa a ser comparável ao do outro dataframe.
dfReceita['CNPJ'] = (dfReceita['CNPJ']).astype(str).copy()

dfReceita.head(5)

,CNPJ,RAZAO_SOCIAL,NOME_FANTASIA,BAIRRO,COD_MUNICIPIO,MUNICIPIO
0,00404850000155,MARTA CRISTINA SOBRAL,MARTA CRISTINA SOBRAL,RIO NOVO,2785,MACEIO
1,00404850000236,MARTA CRISTINA SOBRAL,POSTO LUAR DO FRANCES,POVOADO DO FRANCES,2793,MARECHAL DEODORO
2,00404850000317,MARTA CRISTINA SOBRAL,MARTA CRISTINA SOBRAL,TAB DOS MARTINS,2785,MACEIO
3,00404850000406,MARTA CRISTINA SOBRAL,MARTA CRISTINA SOBRAL,SANTA LUZIA,2833,PENEDO
4,00440239000182,POSTO BOULANGERIE LTDA,POSTO PRAIA,PRADO,2785,MACEIO


<font color='navy'> <strong> O próximo passo é unir os dois conjuntos de dados: receita federal e média de preços dos postos. </font></strong>

In [242]:
#PRIMEIRO INFORMAR QUE O ÍNDICE É O CNPJ
#dfOriginal.set_index("CNPJ", inplace=True) #Já está em str
dfReceita.set_index("CNPJ", inplace=True)

#DEPOIS É POSSÍVEL COMBINAR OS DOIS DATAFRAMES JÁ QUE O ÍNDICE É O MESMO
dfFinal = dfReceita.combine_first(dfOriginal)

#Arredonda valores para 3 casas após a vírgula
dfFinal = dfFinal.round({'VAL_MEDIO_VENDIDO': 3})

#Ordena do menor ao maior valor
dfFinal = dfFinal.sort_values('VAL_MEDIO_VENDIDO').copy()
dfFinal

#Inclui informações manualmente
#CNPJ 33358459000110 é novo e ainda não consta na base de dados da receita

dfFinal.at['33358459000110', 'MUNICIPIO' ] = "MACEIO"
dfFinal.at['33358459000110', 'NOME_FANTASIA'] = "POSTO CASTELO"
dfFinal.at['33358459000110', 'BAIRRO'] = "JACINTINHO"
dfFinal.at['33358459000110', 'COD_MUNICIPIO'] = "2785"
dfFinal.at['33358459000110', 'RAZAO_SOCIAL'] = "AUTO POSTO JACINTINHO LTDA"





dfFinal.head(5)

,BAIRRO,COD_MUNICIPIO,MUNICIPIO,NOME_FANTASIA,RAZAO_SOCIAL,VAL_MEDIO_VENDIDO
CNPJ,,,,,,
33358459000110,JACINTINHO,2785,MACEIO,POSTO CASTELO,AUTO POSTO JACINTINHO LTDA,3.984
22442762000180,ZONA RURAL,2761,IGACI,AUTO POSTO SEGURO,AUTO POSTO SEGURO COM�RCIO DE COMBUST�VEIS LTDA,3.994
32118316000179,CENTRO,2785,MACEIO,POSTO CASTELO,AUTO POSTO MACEIO LTDA,4.051
09178530000107,TABULEIRO DO PINTO,2853,RIO LARGO,AUTO POSTO SAN LORENZZO,SAN LORENZZO COMERCIO E DISTRIBUICAO DE COMBUS...,4.075
20528778000185,FEITOSA,2785,MACEIO,AUTO POSTO CONFIANCA,AUTO POSTO CONFIANCA EIRELI,4.105


In [243]:
#Ordena do menor ao maior valor
dfFinal = dfFinal.sort_values('VAL_MEDIO_VENDIDO').copy()
dfFinal

#Inclui informações manualmente
#CNPJ 33358459000110 é novo e ainda não consta na base de dados da receita

dfFinal.at['33358459000110', 'MUNICIPIO' ] = "MACEIO"
dfFinal.at['33358459000110', 'NOME_FANTASIA'] = "POSTO CASTELO"
dfFinal.at['33358459000110', 'BAIRRO'] = "JACINTINHO"
dfFinal.at['33358459000110', 'COD_MUNICIPIO'] = "2785"
dfFinal.at['33358459000110', 'RAZAO_SOCIAL'] = "AUTO POSTO JACINTINHO LTDA"





<font color='navy'> <strong> Está quase tudo pronto, mas queremos saber também a distância de cada cidade em relação a Maceió. 
    
O que fizemos a seguir foi utilizar a API ( Aplication Programming Interface) do Google Maps para medir a distância de cada uma das cidades até Maceió, levando em consideração a melhor rota rodoviária possível. </font></strong>

In [244]:
#Insere a distância de cada cidade em relação a maceió

import googlemaps 
listaDictDistancias = []
dicionarioDistancias = {'MUNICIPIO': "", 'DISTANCIA': ""}

# Requires API key 
gmaps = googlemaps.Client(key='INSIRA SEU CLIENT KEY') 
import time

#Lista com todas as cidades verificadas
listacidades = dfFinal.MUNICIPIO.unique()
for cidade in listacidades:
        

    origem = "MACEIO"
    destino = cidade
    my_dist = gmaps.distance_matrix(origem + ", Alagoas", destino + ", Alagoas")['rows'][0]['elements'][0] 

    distancia = ""

    try:
    # Printing the result 
        print(destino)
        print(my_dist['distance']['value'])
        distancia = (my_dist['distance']['value'])
    except:
        distancia = "seminfo"
        print("Não encontrado")
        pass

    dicionarioDistancias.update({'MUNICIPIO': destino, 'DISTANCIA': distancia})
    listaDictDistancias.append(dicionarioDistancias.copy())
    time.sleep(0.5)
    print("")

MACEIO
0

IGACI
153705

RIO LARGO
22677

SANTANA DO IPANEMA
209500

CAMPO ALEGRE
93475

SAO MIGUEL DOS CAMPOS
61803

MARECHAL DEODORO
23042

MARAVILHA
234920

SATUBA
22135

BELEM
115845

PILAR
39820

ATALAIA
49997

MATRIZ DE CAMARAGIBE
76129

UNIAO DOS PALMARES
78870

PARIPUEIRA
29574

SAO LUIS DO QUITUNDE
53140

BARRA DE SANTO ANTONIO
38650

ARAPIRACA
132497

JUNQUEIRO
116111

MARIBONDO
90820

MINADOR DO NEGRAO
171681

PALMEIRA DOS INDIOS
139859

CAJUEIRO
73733

PORTO REAL DO COLEGIO
172834

MURICI
54973

BARRA DE SAO MIGUEL
32677

ESTRELA DE ALAGOAS
155939

JOAQUIM GOMES
75243

CAPELA
65543

SAO JOSE DA LAJE
101048

BRANQUINHA
69289

TEOTONIO VILELA
101222

DOIS RIACHOS
190696

BOCA DA MATA
78806

SANTANA DO MUNDAU
105342

ROTEIRO
55019

VICOSA
90077

CRAIBAS
149773

NOVO LINO
101115

SAO SEBASTIAO
131136

CAMPESTRE
118568

ANADIA
97807

POCO DAS TRINCHEIRAS
219972

MARAGOGI
127719

OLIVENCA
206275

LIMOEIRO DE ANADIA
111885

JUNDIA
114903

INHAPI
275174

PAULO JACINTO
108410

PIRANH

<font color='navy'> <strong> Com tudo pronto, basta adicionar a informação de distância à nossa base de dados anterior. </font></strong>

In [245]:
#Atualiza df com a distancia de todas as cidades pra maceió

dfDistancias = pd.DataFrame(listaDictDistancias)
dfDistancias.head(3)


dfFinal2 = dfFinal.merge(dfDistancias, left_on='MUNICIPIO', right_on='MUNICIPIO')
dfFinal2.head(3)

,BAIRRO,COD_MUNICIPIO,MUNICIPIO,NOME_FANTASIA,RAZAO_SOCIAL,VAL_MEDIO_VENDIDO,DISTANCIA
0,JACINTINHO,2785,MACEIO,POSTO CASTELO,AUTO POSTO JACINTINHO LTDA,3.984,0
1,CENTRO,2785,MACEIO,POSTO CASTELO,AUTO POSTO MACEIO LTDA,4.051,0
2,FEITOSA,2785,MACEIO,AUTO POSTO CONFIANCA,AUTO POSTO CONFIANCA EIRELI,4.105,0


<font color='navy'> <strong> Pra finalizar é só salvar em CSV e podemos exportar para uma planilha eletrônica ou ferramentas específicas de visualização de dados. </font></strong>

In [246]:
import csv
dfFinal2.to_csv("combustiveis_alagoas_municipios_distancias.csv", sep=',', quotechar='"',index=False, quoting=csv.QUOTE_ALL)